In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [105]:
label_encoder = LabelEncoder()
file_path = r"D:\Coding_Dev\College\SEM_6\ML\Assets\breast_cancer_wisconsin_original\breast-cancer-wisconsin.data"
columns = ["Feature0","Feature1", "Feature2", "Feature3", "Feature4", "Feature5", "Feature6", "Feature7", "Feature8", "Feature9", "Class_Label"]
data = pd.read_csv(file_path, header=None, sep=",", names=columns)
data.drop("Feature0", axis=1, inplace=True)
data.replace('?', pd.NA, inplace=True)
data.dropna(inplace=True)

In [133]:
features = data.drop("Class_Label", axis=1)
labels = data["Class_Label"]

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

In [134]:
def calculate_conditional_probs(data, feature, class_labels):
    conditional_probs = {}
    
    for label in class_labels:
        subset = data[y_train == label]
        hist, _ = np.histogram(subset[feature].astype(float), bins=10, density=True)
        conditional_probs[label] = hist / hist.sum()
        
    return conditional_probs

class_labels = y_train.unique()
conditional_probs = {}

for feature in X_train.columns:
    conditional_probs[feature] = calculate_conditional_probs(X_train, feature, class_labels)


In [135]:
prior_probs = y_train.value_counts(normalize=True).to_dict()

In [136]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

nb_classifier = GaussianNB()
nb_classifier.fit(X_train_scaled, y_train)


predictions = nb_classifier.predict(X_test_scaled)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9658536585365853


In [137]:
def classify_sample(sample, conditional_probs, prior_probs):
    class_scores = {}
    
    for label in class_labels:
        score = np.log(prior_probs[label])
        
        for feature, value in sample.items():
            bin_index = np.searchsorted(np.linspace(1, 11, num=11), value) - 1
            score += np.log(conditional_probs[feature][label][bin_index])
            
        class_scores[label] = score
        
    return max(class_scores, key=class_scores.get)

predictions = X_test.apply(lambda sample: classify_sample(sample, conditional_probs, prior_probs), axis=1)


C:\Users\barar\AppData\Local\Temp\ipykernel_22544\3016662624.py:9: RuntimeWarning: divide by zero encountered in log
  score += np.log(conditional_probs[feature][label][bin_index])


In [138]:
mean_accuracy = (predictions == y_test).mean()
print("Accuracy:", accuracy)

Accuracy: 0.9658536585365853
